In [1]:
# !pip install -q transformers accelerate datasets bitsandbytes

In [2]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
df = pd.read_csv('/content/project_dataset.csv')
df.head(10)

,Feedback,Category,Sentiment
0,The dining environment promotes social interac...,Mess,Neutral
1,College doesn't organize regular sports and at...,Others,Negative
2,Teaching load is distributed among faculty tho...,Faculty,Neutral
3,College administration consists of various dep...,Administration,Neutral
4,Student success initiatives are coordinated th...,Administration,Neutral
5,Eco-friendly initiatives including waste segre...,Hostel,Positive
6,Medical facilities don't provide health educat...,Facilities,Negative
7,Administrative forms are available online thro...,Administration,Neutral
8,Limited practical training opportunities in la...,Academics,Negative
9,Academic partnerships with other institutions ...,Academics,Negative


In [4]:
train_data, test_data = train_test_split(
    df, test_size=0.1, random_state=42, stratify=df[['Category', 'Sentiment']]
)

In [5]:
# For Category Classification (DeBERTa-v3)
cat_model_id = "microsoft/deberta-v3-base"
cat_tokenizer = AutoTokenizer.from_pretrained(cat_model_id)
cat_model = AutoModelForSequenceClassification.from_pretrained(
    cat_model_id, num_labels=len(train_data['Category'].unique())
).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

In [6]:
# For Sentiment Analysis (RoBERTa-base)
sent_model_id = "roberta-base"
sent_tokenizer = AutoTokenizer.from_pretrained(sent_model_id)
sent_model = AutoModelForSequenceClassification.from_pretrained(
    sent_model_id, num_labels=len(train_data['Sentiment'].unique())
).to("cuda")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
cat2id = {cat: i for i, cat in enumerate(sorted(train_data['Category'].unique()))}
id2cat = {i: cat for cat, i in cat2id.items()}
sent2id = {sent: i for i, sent in enumerate(sorted(train_data['Sentiment'].unique()))}
id2sent = {i: sent for sent, i in sent2id.items()}

train_data['cat_id'] = train_data['Category'].map(cat2id)
train_data['sent_id'] = train_data['Sentiment'].map(sent2id)
test_data['cat_id'] = test_data['Category'].map(cat2id)
test_data['sent_id'] = test_data['Sentiment'].map(sent2id)

In [8]:
from torch.utils.data import Dataset, DataLoader

class FeedbackDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

cat_train_ds = FeedbackDataset(
    train_data['Feedback'].tolist(), train_data['cat_id'].tolist(), cat_tokenizer
)
cat_test_ds = FeedbackDataset(
    test_data['Feedback'].tolist(), test_data['cat_id'].tolist(), cat_tokenizer
)
sent_train_ds = FeedbackDataset(
    train_data['Feedback'].tolist(), train_data['sent_id'].tolist(), sent_tokenizer
)
sent_test_ds = FeedbackDataset(
    test_data['Feedback'].tolist(), test_data['sent_id'].tolist(), sent_tokenizer
)


In [9]:
from transformers import TrainingArguments, Trainer

# Category classification
cat_training_args = TrainingArguments(
    output_dir='./cat_results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)
def compute_metrics_cat(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": (preds == labels).mean()}

cat_trainer = Trainer(
    model=cat_model,
    args=cat_training_args,
    train_dataset=cat_train_ds,
    eval_dataset=cat_test_ds,
    compute_metrics=compute_metrics_cat,
)

cat_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanmoysaha (tanmoysaha-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.744019,0.781513
2,No log,0.334705,0.890756
3,No log,0.345608,0.890756


TrainOutput(global_step=402, training_loss=0.8248118689997279, metrics={'train_runtime': 236.9611, 'train_samples_per_second': 13.559, 'train_steps_per_second': 1.696, 'total_flos': 211357233287424.0, 'train_loss': 0.8248118689997279, 'epoch': 3.0})

In [10]:
# Sentiment classification
# 38b7e26aac99a74603014cb79996f89379baf48e
sent_training_args = TrainingArguments(
    output_dir='./sent_results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)
def compute_metrics_sent(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": (preds == labels).mean()}

sent_trainer = Trainer(
    model=sent_model,
    args=sent_training_args,
    train_dataset=sent_train_ds,
    eval_dataset=sent_test_ds,
    compute_metrics=compute_metrics_sent,
)

sent_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.106627,0.983193
2,No log,0.116618,0.983193
3,No log,0.103910,0.983193


TrainOutput(global_step=402, training_loss=0.15931621950064132, metrics={'train_runtime': 102.8982, 'train_samples_per_second': 31.225, 'train_steps_per_second': 3.907, 'total_flos': 211345852790016.0, 'train_loss': 0.15931621950064132, 'epoch': 3.0})

In [11]:
cat_preds, sent_preds = [], []
cat_model.eval(); sent_model.eval()

for feedback in tqdm(test_data['Feedback'].tolist()):
    # Category
    inputs = cat_tokenizer(feedback, return_tensors="pt", truncation=True, max_length=128).to("cuda")
    with torch.no_grad():
        outputs = cat_model(**inputs)
        pred = outputs.logits.argmax(dim=1).item()
        cat_preds.append(id2cat[pred])
    # Sentiment
    inputs = sent_tokenizer(feedback, return_tensors="pt", truncation=True, max_length=128).to("cuda")
    with torch.no_grad():
        outputs = sent_model(**inputs)
        pred = outputs.logits.argmax(dim=1).item()
        sent_preds.append(id2sent[pred])

100%|██████████| 119/119 [00:04<00:00, 26.54it/s]


In [12]:
print("\n📌 Category Classification Report:")
print(classification_report(test_data['Category'], cat_preds))
print(f"Accuracy: {accuracy_score(test_data['Category'], cat_preds):.2%}\n")

print("📌 Sentiment Analysis Report:")
print(classification_report(test_data['Sentiment'], sent_preds))
print(f"Accuracy: {accuracy_score(test_data['Sentiment'], sent_preds):.2%}")


📌 Category Classification Report:
                precision    recall  f1-score   support

     Academics       0.85      0.65      0.73        17
Administration       1.00      1.00      1.00        17
    Facilities       1.00      0.94      0.97        17
       Faculty       1.00      1.00      1.00        17
        Hostel       0.88      0.82      0.85        17
          Mess       0.94      1.00      0.97        17
        Others       0.64      0.82      0.72        17

      accuracy                           0.89       119
     macro avg       0.90      0.89      0.89       119
  weighted avg       0.90      0.89      0.89       119

Accuracy: 89.08%

📌 Sentiment Analysis Report:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        42
     Neutral       0.97      0.97      0.97        35
    Positive       0.98      0.98      0.98        42

    accuracy                           0.98       119
   macro avg       0.98    

In [13]:
def predict_review(feedback_text):
    # Predict Category
    cat_inputs = cat_tokenizer(feedback_text, return_tensors="pt", truncation=True, max_length=128).to("cuda")
    with torch.no_grad():
        cat_outputs = cat_model(**cat_inputs)
        cat_pred_id = cat_outputs.logits.argmax(dim=1).item()
        cat_pred = id2cat[cat_pred_id]

    # Predict Sentiment
    sent_inputs = sent_tokenizer(feedback_text, return_tensors="pt", truncation=True, max_length=128).to("cuda")
    with torch.no_grad():
        sent_outputs = sent_model(**sent_inputs)
        sent_pred_id = sent_outputs.logits.argmax(dim=1).item()
        sent_pred = id2sent[sent_pred_id]

    # Chatbot-style response
    print("🤖 Review Prediction Bot")
    print(f"📝 Feedback: {feedback_text}")
    print(f"📂 Predicted Category: {cat_pred}")
    print(f"💬 Predicted Sentiment: {sent_pred}")

In [14]:
# Example usage:
user_review = input("Enter your feedback: ")
predict_review(user_review)

Enter your feedback: Living in the hostel has been a great experience. Clean washrooms and regular maintenance make it comfortable.
🤖 Review Prediction Bot
📝 Feedback: Living in the hostel has been a great experience. Clean washrooms and regular maintenance make it comfortable.
📂 Predicted Category: Administration
💬 Predicted Sentiment: Positive
